In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()     # M-MBulge Relation            (MMB)

# shape = (150, 151, 152)
shape = (30, 31, 32)
shape = None

print(f"{shape=}")
sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=shape)
print(f"{sam.shape=}")

In [ ]:
print(sam.shape)
breaker()


In [ ]:
# Choose observed GW-Frequency bins based on nyquist sampling
fobs = utils.nyquist_freqs(10.0*YR, 0.1*YR)
# fobs = utils.nyquist_freqs(10.0*YR, 1.0*YR)
# fobs = utils.nyquist_freqs(10.0*YR, 1.0*YR)
# print(fobs.size, fobs.min(), fobs.max())
fobs = kale.utils.spacing(fobs, scale='log', num=fobs.size)
# print(fobs.size, fobs.min(), fobs.max())

In [ ]:
vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(
        sam, holo.evolution.Hard_GW, fobs=fobs,
        sample_threshold=5.0, cut_below_mass=3e7, limit_merger_time=4*GYR,
)

In [ ]:
# gwf_freqs, gwf, gwb = holo.sam._gws_from_samples(vals, weights, fobs)

In [ ]:
gwb_smooth = sam.gwb(fobs, realize=False)
gwb_rough = sam.gwb(fobs, realize=10)

In [ ]:
SAMP_NREALS = 10
gwb = np.zeros((fobs.size-1, SAMP_NREALS))
gwf = np.zeros((fobs.size-1, SAMP_NREALS))
for ii in utils.tqdm(range(SAMP_NREALS)):
    gwf_freqs, gwf[:, ii], gwb[:, ii] = holo.sam.sampled_gws_from_sam(
        sam, fobs=fobs, hard=holo.evolution.Hard_GW, 
        sample_threshold=3.0, cut_below_mass=1e6, limit_merger_time=None,
    )
    # gwb[:, ii] = np.sqrt(_gwb[:]**2 + gwf**2)

In [ ]:
fig, ax = plot.figax()

def plot_gwb(xx, yy, **kwargs):
    cc, = ax.plot(xx, np.median(yy, axis=-1), ls='--', **kwargs)
    cc = cc.get_color()
    temp = np.percentile(yy, [25, 75], axis=-1)
    ax.fill_between(xx, *temp, color=cc, alpha=0.25)    
    return

xx = kale.utils.midpoints(fobs) * YR   # [1/sec] ==> [1/yr]

amp = 10e-16
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)

ax.plot(xx, gwb_smooth, 'b:')
plot_gwb(xx, gwb_rough, label='roughed')
if np.ndim(gwb) == 1:
    ax.plot(xx, gwb, 'k-', alpha=0.25)
else:
    plot_gwb(xx, np.sqrt(gwb**2 + gwf**2), label='sampled')

plt.legend()
plt.show()
# fig.savefig('temp.png')

# Manual

In [ ]:
vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(
        sam, holo.evolution.Hard_GW, fobs=fobs,
        sample_threshold=5.0, cut_below_mass=None, limit_merger_time=None,
)

In [ ]:
kale.utils.jshape(edges)

In [ ]:
counts, _ = np.histogramdd(vals.T, bins=edges, weights=weights)
print(counts.shape)

In [ ]:
import zcode
import zcode.math as zmath

In [ ]:
LOG = True
idx = (None, 0, 0, None)
print(counts[idx].squeeze().shape)
print(mass[idx].squeeze().shape)

fig, axes = plt.subplots(figsize=[10, 4], ncols=2)
extr = zmath.minmax(mass, prev=zmath.minmax(counts), limit=[0.5, None])
if LOG:
    extr = np.log10(extr)
smap = plot.smap(extr)
print(f"{extr=}")
for ii, (ax, qq) in enumerate(zip(axes, [counts, mass])):
    qq = qq[idx].squeeze()
    if LOG:
        qq = np.log10(qq)
    ax.pcolormesh(qq, cmap=smap.cmap, norm=smap.norm)
    
plt.colorbar(smap)
plt.show()

In [ ]:
mc = utils.chirp_mass(*utils.m1m2_from_mtmr(vals[0], vals[1]))
rz = vals[2]
fo = vals[3]
frst = utils.frst_from_fobs(fo, rz)
dc = cosmo.comoving_distance(rz).cgs.value
hs = utils.gw_strain_source(mc, dc, frst)
print(hs.shape, fo.shape, weights.shape)

In [ ]:
idx = np.argsort(fo)
fo = fo[idx]
hs = hs[idx]
weights = weights[idx]

idx = np.digitize(fo, fobs) - 1
gwb = np.zeros(fobs.size-1)

for ii in utils.tqdm(range(gwb.size)):
    sel = (idx == ii)
    temp = weights[sel] * (hs[sel] ** 2)
    gwb[ii] = np.sum(temp)
    
gwb = np.sqrt(gwb)

In [ ]:
fig, ax = plot.figax()

def plot_gwb(xx, yy, **kwargs):
    cc, = ax.plot(xx, np.median(yy, axis=-1), ls='--', **kwargs)
    cc = cc.get_color()
    temp = np.percentile(yy, [25, 75], axis=-1)
    ax.fill_between(xx, *temp, color=cc, alpha=0.25)    
    return

xx = kale.utils.midpoints(fobs) * YR   # [1/sec] ==> [1/yr]

amp = 10e-16
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)
ax.plot(xx, gwb, 'k-', alpha=0.25)

ax.plot(xx, gwb_smooth, 'b:')
plot_gwb(xx, gwb_rough, label='roughed')
# plot_gwb(xx, gwb, label='sampled')

plt.legend()
plt.show()

# Number of Sources vs. Frequency

In [ ]:
NUM_REALS = 10
vals = []
weights = []
for ii in holo.utils.tqdm(range(NUM_REALS)):
    _vals, _weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(
            sam, holo.evolution.Hard_GW, fobs=fobs,
            sample_threshold=5.0, cut_below_mass=None, limit_merger_time=None,
    )
    vals.append(_vals)
    weights.append(_weights)

In [ ]:
edges[0].shape

In [ ]:
num_fobs = fobs.size
# MASS_MIN = 1.0e9 * MSOL
MASS_MIN = 0.0
num = np.zeros((num_fobs, NUM_REALS))

for ii in holo.utils.tqdm(range(NUM_REALS)):
    fo = vals[ii][-1]
    idx = np.digitize(fo, fobs) - 1
    for jj in range(num_fobs):
        sel = (idx == jj) & (vals[ii][0] > MASS_MIN)
        num[jj, ii] = np.sum(weights[ii][sel])
        
sel = (kale.utils.midpoints(edges[0]) > MASS_MIN)
idx = list(np.arange(mass.ndim))
idx.pop(mass.ndim - 1)
sam_num = np.sum(mass[sel], axis=tuple(idx))

In [ ]:
def get_steps(xx, yy):
    aa = [[xx[ii], xx[ii+1]] for ii in range(fobs.size-1)]
    bb = [[yy[ii], yy[ii]] for ii in range(fobs.size-1)]
    aa = np.array(aa).flatten()
    bb = np.array(bb).flatten()
    return aa, bb

In [ ]:
fig, ax = plot.figax()
xx = fobs*YR
# ax.plot(xx, num)
ax.plot(kale.utils.midpoints(xx), num.mean(axis=-1)[:-1], 'b-')
ax.plot(kale.utils.midpoints(xx), sam_num, 'r--')

aa, bb = get_steps(xx, sam_num)
ax.plot(aa, bb, 'r:')
plt.show()